# Deep Lake | ディープ レイク

> [Deep Lake](https://www.activeloop.ai)は、AIアプリケーション構築のためのマルチモーダルデータベースです。[Deep Lake](https://github.com/activeloopai/deeplake)はAI向けのデータベースで、ベクトル、画像、テキスト、動画などを保存できます。LLMs/LangChainと組み合わせて使用することができ、あらゆるAIデータの保存、クエリ、バージョン管理、可視化を行うことができます。また、PyTorch/TensorFlowにリアルタイムでデータをストリーミングすることも可能です。
>
> > [Deep Lake](https://www.activeloop.ai) is a multimodal database for building AI applications
> > [Deep Lake](https://github.com/activeloopai/deeplake) is a database for AI.
> > Store Vectors, Images, Texts, Videos, etc. Use with LLMs/LangChain. Store, query, version,
> > & visualize any AI data. Stream data in real time to PyTorch/TensorFlow.

このノートブックでは、`Deep Lake` ベクターストアを使用して `SelfQueryRetriever` をデモします。

> In the notebook, we'll demo the `SelfQueryRetriever` wrapped around a `Deep Lake` vector store.



## Creating a Deep Lake vector store | Deep Lakeベクトルストアの作成

まず、Deep Lakeベクトルストアを作成し、いくつかのデータを投入して初期化します。映画の要約を含む小さなデモ文書セットを作成しました。

> First we'll want to create a Deep Lake vector store and seed it with some data. We've created a small demo set of documents that contain summaries of movies.

**注意:** self-query retrieverを使用するには、`lark`がインストールされている必要があります（`pip install lark`でインストールできます）。さらに、`deeplake`パッケージも必要です。

> **Note:** The self-query retriever requires you to have `lark` installed (`pip install lark`). We also need the `deeplake` package.



In [ ]:
# !pip install lark


In [2]:
# in case if some queries fail consider installing libdeeplake manually
# !pip install libdeeplake


`OpenAIEmbeddings`を使用するためには、OpenAI APIキーを取得する必要があります。

> We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.



In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["ACTIVELOOP_TOKEN"] = getpass.getpass("Activeloop token:")

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import DeepLake

embeddings = OpenAIEmbeddings()

In [6]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "science fiction",
            "rating": 9.9,
        },
    ),
]
username_or_org = "<USERNAME_OR_ORG>"
vectorstore = DeepLake.from_documents(
    docs,
    embeddings,
    dataset_path=f"hub://{username_or_org}/self_queery",
    overwrite=True,
)

Your Deep Lake dataset has been successfully created!


/

Dataset(path='hub://adilkhan/self_queery', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (6, 1536)  float32   None   
    id        text      (6, 1)      str     None   
 metadata     json      (6, 1)      str     None   
   text       text      (6, 1)      str     None   


## Creating our self-querying retriever | 自己問い合わせ型リトリーバーの作成

これで、私たちのリトリーバーをインスタンス化することができます。これを行うには、ドキュメントがサポートしているメタデータフィールドに関する情報と、ドキュメントの内容の簡潔な説明を事前に提供する必要があります。

> Now we can instantiate our retriever. To do this we'll need to provide some information upfront about the metadata fields that our documents support and a short description of the document contents.



In [7]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

## Testing it out | 試してみる

それでは、実際に私たちのリトリーバーを使ってみましょう！

> And now we can try actually using our retriever!



In [8]:
# This example only specifies a relevant query
retriever.get_relevant_documents("What are some movies about dinosaurs")

/home/ubuntu/langchain_activeloop/langchain/libs/langchain/langchain/chains/llm.py:279: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='dinosaur' filter=None limit=None


[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'year': 1993, 'rating': 7.7, 'genre': 'science fiction'}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'year': 1995, 'genre': 'animated'}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'year': 1979, 'rating': 9.9, 'director': 'Andrei Tarkovsky', 'genre': 'science fiction'}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'year': 2006, 'director': 'Satoshi Kon', 'rating': 8.6})]

In [9]:
# This example only specifies a filter
retriever.get_relevant_documents("I want to watch a movie rated higher than 8.5")

# in case if this example errored out, consider installing libdeeplake manually: `pip install libdeeplake`, and then restart notebook.

query=' ' filter=Comparison(comparator=<Comparator.GT: 'gt'>, attribute='rating', value=8.5) limit=None


[Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'year': 2006, 'director': 'Satoshi Kon', 'rating': 8.6}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'year': 1979, 'rating': 9.9, 'director': 'Andrei Tarkovsky', 'genre': 'science fiction'})]

In [10]:
# This example specifies a query and a filter
retriever.get_relevant_documents("Has Greta Gerwig directed any movies about women")

query='women' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='director', value='Greta Gerwig') limit=None


[Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'year': 2019, 'director': 'Greta Gerwig', 'rating': 8.3})]

In [11]:
# This example specifies a composite filter
retriever.get_relevant_documents(
    "What's a highly rated (above 8.5) science fiction film?"
)

query=' ' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='rating', value=8.5), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genre', value='science fiction')]) limit=None


[Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'year': 1979, 'rating': 9.9, 'director': 'Andrei Tarkovsky', 'genre': 'science fiction'})]

In [12]:
# This example specifies a query and composite filter
retriever.get_relevant_documents(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

query='toys' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.GT: 'gt'>, attribute='year', value=1990), Comparison(comparator=<Comparator.LT: 'lt'>, attribute='year', value=2005), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genre', value='animated')]) limit=None


[Document(page_content='Toys come alive and have a blast doing so', metadata={'year': 1995, 'genre': 'animated'})]

## Filter k | フィルター k

さらに、self query retrieverを使用して、取得するドキュメントの数`k`を指定することもできます。

> We can also use the self query retriever to specify `k`: the number of documents to fetch.

これを行うには、コンストラクタに `enable_limit=True` を渡すことで実現できます。

> We can do this by passing `enable_limit=True` to the constructor.



In [13]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    verbose=True,
)

In [14]:
# This example only specifies a relevant query
retriever.get_relevant_documents("what are two movies about dinosaurs")

query='dinosaur' filter=None limit=2


[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'year': 1993, 'rating': 7.7, 'genre': 'science fiction'}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'year': 1995, 'genre': 'animated'})]